In [1]:
import wandb
wandb.login(key="5fb34431b405eb21dc0f263e5b3cf2c15fdc7471")

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: ma24m004 (ma24m004-iit-madras) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
import wandb
from torch.nn.utils.rnn import pad_sequence

# Encoder
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hidden_dim, n_layers, cell_type, dropout):
        super().__init__()
        self.embedding = nn.Embedding(input_dim, emb_dim)
        self.rnn = getattr(nn, cell_type)(emb_dim, hidden_dim, n_layers, dropout=dropout, batch_first=True)
        self.dropout = nn.Dropout(dropout)

    def forward(self, src):
        embedded = self.dropout(self.embedding(src))
        outputs, hidden = self.rnn(embedded)
        if isinstance(hidden, tuple):  # LSTM
            return hidden[0].contiguous(), hidden[1].contiguous()
        return hidden.contiguous(), None  # RNN, GRU

# Decoder
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hidden_dim, n_layers, cell_type, dropout):
        super().__init__()
        self.output_dim = output_dim
        self.embedding = nn.Embedding(output_dim, emb_dim)
        self.rnn = getattr(nn, cell_type)(emb_dim, hidden_dim, n_layers, dropout=dropout, batch_first=True)
        self.fc_out = nn.Linear(hidden_dim, output_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, input, hidden, cell=None):
        input = input.unsqueeze(1)
        embedded = self.dropout(self.embedding(input))
        if cell is not None:
            hidden = hidden.contiguous()
            cell = cell.contiguous()
            output, (hidden, cell) = self.rnn(embedded, (hidden, cell))
        else:
            hidden = hidden.contiguous()
            output, hidden = self.rnn(embedded, hidden)
        prediction = self.fc_out(output.squeeze(1))
        return prediction, hidden, cell

# Seq2Seq Model with Beam Search
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device

    def forward(self, src, trg, teacher_forcing_ratio=0.5):
        batch_size = src.shape[0]
        trg_len = trg.shape[1]
        trg_vocab_size = self.decoder.output_dim

        outputs = torch.zeros(batch_size, trg_len, trg_vocab_size).to(self.device)
        hidden, cell = self.encoder(src)

        input = trg[:, 0]
        for t in range(1, trg_len):
            output, hidden, cell = self.decoder(input, hidden, cell)
            outputs[:, t] = output
            teacher_force = np.random.random() < teacher_forcing_ratio
            top1 = output.argmax(1)
            input = trg[:, t] if teacher_force else top1
        return outputs

    def predict(self, src, max_len=30, beam_size=3):
        self.eval()
        batch_size = src.size(0)
        hidden, cell = self.encoder(src)
        outputs = []

        for i in range(batch_size):
            h = hidden[:, i:i+1].contiguous()
            c = cell[:, i:i+1].contiguous() if cell is not None else None
            beams = [(torch.tensor([1], device=self.device), 0.0, h, c)]  # [sequence, score, hidden, cell]
            for _ in range(max_len):
                new_beams = []
                for seq, score, h, c in beams:
                    input = seq[-1].unsqueeze(0)
                    output, h_new, c_new = self.decoder(input, h, c)
                    probs = torch.log_softmax(output, dim=1).squeeze(0)
                    topk = torch.topk(probs, beam_size)
                    for idx, prob in zip(topk.indices, topk.values):
                        new_seq = torch.cat([seq, idx.unsqueeze(0)])
                        new_beams.append((new_seq, score + prob.item(), h_new.contiguous(), c_new.contiguous() if c_new is not None else None))
                beams = sorted(new_beams, key=lambda x: x[1], reverse=True)[:beam_size]
                if beams[0][0][-1].item() == 2:  # Stop if <EOS>
                    break
            outputs.append(beams[0][0][1:])
        return outputs

# Dataset
class DakshinaDataset(Dataset):
    def __init__(self, data, input_vocab, output_vocab):
        self.data = data
        self.input_vocab = input_vocab
        self.output_vocab = output_vocab

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        src = [self.input_vocab.get(c, self.input_vocab['<UNK>']) for c in self.data.iloc[idx, 1]] + [self.input_vocab['<EOS>']]
        tgt = [self.output_vocab['<SOS>']] + [self.output_vocab.get(c, self.output_vocab['<UNK>']) for c in self.data.iloc[idx, 0]] + [self.output_vocab['<EOS>']]
        return torch.tensor(src), torch.tensor(tgt)

# Vocab Creation
def create_vocab(data, column):
    vocab = {'<PAD>': 0, '<SOS>': 1, '<EOS>': 2, '<UNK>': 3}
    for seq in data[column]:
        if pd.notna(seq):
            for char in seq:
                if char not in vocab:
                    vocab[char] = len(vocab)
    return vocab

# Collate
def pad_collate(batch):
    src_batch, tgt_batch = zip(*batch)
    src_padded = pad_sequence(src_batch, batch_first=True, padding_value=0)
    tgt_padded = pad_sequence(tgt_batch, batch_first=True, padding_value=0)
    return src_padded, tgt_padded


In [3]:

# Train
def train_model(config=None):
    with wandb.init(config=config):
        config = wandb.config
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

        # Load data
        train_data = pd.read_csv('/kaggle/input/dakshina/dakshina_dataset_v1.0/hi/lexicons/hi.translit.sampled.train.tsv', sep='\t', header=None, dtype=str).dropna()
        dev_data = pd.read_csv('/kaggle/input/dakshina/dakshina_dataset_v1.0/hi/lexicons/hi.translit.sampled.dev.tsv', sep='\t', header=None, dtype=str).dropna()

        # Create vocabularies
        input_vocab = create_vocab(train_data, 1)  # English (source)
        output_vocab = create_vocab(train_data, 0)  # Hindi (target)

        # Prepare datasets and loaders
        train_dataset = DakshinaDataset(train_data, input_vocab, output_vocab)
        dev_dataset = DakshinaDataset(dev_data, input_vocab, output_vocab)
        train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, collate_fn=pad_collate)
        dev_loader = DataLoader(dev_dataset, batch_size=32, shuffle=False, collate_fn=pad_collate)

        # Initialize model
        encoder = Encoder(len(input_vocab), config.emb_dim, config.hidden_dim, config.num_layers, config.cell_type, config.dropout)
        decoder = Decoder(len(output_vocab), config.emb_dim, config.hidden_dim, config.num_layers, config.cell_type, config.dropout)
        model = Seq2Seq(encoder, decoder, device).to(device)

        # Loss and optimizer
        optimizer = optim.Adam(model.parameters(), lr=0.001)
        criterion = nn.CrossEntropyLoss(ignore_index=0)

        best_val_accuracy = 0

        # Training loop
        for epoch in range(config.epochs):
            model.train()
            train_loss, train_correct, train_total = 0, 0, 0
            for src, tgt in train_loader:
                src, tgt = src.to(device), tgt.to(device)
                optimizer.zero_grad()
                output = model(src, tgt)
                output = output[:, 1:].reshape(-1, output.shape[-1])
                tgt = tgt[:, 1:].reshape(-1)
                loss = criterion(output, tgt)
                loss.backward()
                optimizer.step()

                train_loss += loss.item()
                preds = output.argmax(dim=1)
                non_pad_mask = tgt != 0
                train_correct += (preds == tgt)[non_pad_mask].sum().item()
                train_total += non_pad_mask.sum().item()

            train_accuracy = train_correct / train_total

            # Validation
            model.eval()
            val_loss, val_correct, val_total = 0, 0, 0
            with torch.no_grad():
                for src, tgt in dev_loader:
                    src, tgt = src.to(device), tgt.to(device)
                    output = model(src, tgt, teacher_forcing_ratio=0)
                    output = output[:, 1:].reshape(-1, output.shape[-1])
                    tgt = tgt[:, 1:].reshape(-1)
                    loss = criterion(output, tgt)
                    val_loss += loss.item()
                    preds = output.argmax(dim=1)
                    non_pad_mask = tgt != 0
                    val_correct += (preds == tgt)[non_pad_mask].sum().item()
                    val_total += non_pad_mask.sum().item()

            val_accuracy = val_correct / val_total
            patience_counter = 0
            # Save best model
            if val_accuracy > best_val_accuracy:
                best_val_accuracy = val_accuracy
                torch.save(model.state_dict(), '/kaggle/working/best_model.pt')
            else:
                patience_counter += 1
                if patience_counter >= 3:
                    print("Early stopping triggered.")
                    break

            # Log metrics to wandb
            wandb.log({
                'epoch': epoch,
                'train_loss': train_loss / len(train_loader),
                'val_loss': val_loss / len(dev_loader),
                'train_accuracy': train_accuracy,
                'val_accuracy': val_accuracy
            })

            # Log sample predictions
            src_sample, tgt_sample = next(iter(dev_loader))
            src_sample, tgt_sample = src_sample.to(device), tgt_sample.to(device)
            preds = model.predict(src_sample[:5], beam_size=config.beam_size)

            inv_input_vocab = {v: k for k, v in input_vocab.items()}
            inv_output_vocab = {v: k for k, v in output_vocab.items()}
            table = wandb.Table(columns=["Input", "Target", "Prediction"])
            for i in range(len(preds)):
                input_str = ''.join([inv_input_vocab.get(id.item(), '?') for id in src_sample[i] if id.item() not in [0, input_vocab['<EOS>']]])
                target_str = ''.join([inv_output_vocab.get(id.item(), '?') for id in tgt_sample[i] if id.item() not in [0, output_vocab['<EOS>']]])
                pred_str = ''.join([inv_output_vocab.get(id.item(), '?') for id in preds[i]])
                table.add_data(input_str, target_str, pred_str)
            wandb.log({"Predictions": table})


In [5]:

# Sweep
sweep_config = {
    'method': 'random',
    'metric': {'name': 'val_accuracy', 'goal': 'maximize'},
    'parameters': {
        'emb_dim': {'values': [16, 32, 64, 256]},
        'hidden_dim': {'values': [16, 32, 64, 256]},
        'num_layers': {'values': [1, 2, 3]},
        'cell_type': {'values': ['RNN', 'GRU', 'LSTM']},
        'dropout': {'values': [0, 0.2, 0.3]},
        'epochs': {'values': [5, 10]},
        'beam_size': {'values': [1, 3, 5]}
    }
}

sweep_id = wandb.sweep(sweep_config, project='DL_ASSIGNMENT_3_RNN')
wandb.agent(sweep_id, train_model, count=60)
wandb.finish()

Create sweep with ID: pd9x2ajs
Sweep URL: https://wandb.ai/ma24m004-iit-madras/DL_ASSIGNMENT_3_RNN/sweeps/pd9x2ajs


wandb: Agent Starting Run: krv2nmsb with config:
wandb: 	beam_size: 1
wandb: 	cell_type: GRU
wandb: 	dropout: 0
wandb: 	emb_dim: 16
wandb: 	epochs: 10
wandb: 	hidden_dim: 16
wandb: 	num_layers: 3
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


epoch,▁▂▃▃▄▅▆▆▇█
train_accuracy,▁▂▄▅▅▆▇▇██
train_loss,█▆▅▄▃▃▂▂▁▁
val_accuracy,▁▂▃▄▅▆▇▇██
val_loss,█▇▅▄▃▃▂▂▁▁
epoch,9
train_accuracy,0.51096
train_loss,1.70208
val_accuracy,0.46767
val_loss,1.88986


wandb: Agent Starting Run: daggptac with config:
wandb: 	beam_size: 5
wandb: 	cell_type: RNN
wandb: 	dropout: 0.2
wandb: 	emb_dim: 32
wandb: 	epochs: 5
wandb: 	hidden_dim: 16
wandb: 	num_layers: 3


epoch,▁▃▅▆█
train_accuracy,▁▅▆▇█
train_loss,█▃▂▂▁
val_accuracy,▁▄███
val_loss,█▅▁▁▁
epoch,4
train_accuracy,0.24528
train_loss,2.86821
val_accuracy,0.21969
val_loss,3.021


...

epoch,▁▂▃▃▄▅▆▆▇█
train_accuracy,▁▅▆▇▇▇████
train_loss,█▄▃▂▂▂▁▁▁▁
val_accuracy,▁▅▆▇▇█████
val_loss,█▃▂▁▁▁▁▁▁▁
epoch,9
train_accuracy,0.87349
train_loss,0.41591
val_accuracy,0.73123
val_loss,1.06099


wandb: Agent Starting Run: zjw5zoye with config:
wandb: 	beam_size: 5
wandb: 	cell_type: LSTM
wandb: 	dropout: 0
wandb: 	emb_dim: 64
wandb: 	epochs: 10
wandb: 	hidden_dim: 32
wandb: 	num_layers: 1


epoch,▁▂▃▃▄▅▆▆▇█
train_accuracy,▁▃▄▅▆▆▇▇██
train_loss,█▆▄▄▃▂▂▁▁▁
val_accuracy,▁▃▄▅▆▆▇▇██
val_loss,█▆▅▄▃▂▂▂▁▁
epoch,9
train_accuracy,0.57864
train_loss,1.40791
val_accuracy,0.48845
val_loss,1.74818


wandb: Agent Starting Run: iuvhz5nx with config:
wandb: 	beam_size: 5
wandb: 	cell_type: GRU
wandb: 	dropout: 0.3
wandb: 	emb_dim: 64
wandb: 	epochs: 10
wandb: 	hidden_dim: 64
wandb: 	num_layers: 3


epoch,▁▂▃▃▄▅▆▆▇█
train_accuracy,▁▄▆▆▇▇▇███
train_loss,█▄▃▂▂▂▁▁▁▁
val_accuracy,▁▄▆▆▇▇████
val_loss,█▄▃▂▂▂▁▁▁▁
epoch,9
train_accuracy,0.70723
train_loss,0.94979
val_accuracy,0.65576
val_loss,1.15449


In [ ]:
import random
import torch
from torch.utils.data import DataLoader
import pandas as pd
import wandb
from IPython.display import display, HTML

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Initialize Wandb
wandb.init(project="DL_ASSIGNMENT_3_RNN", name="vanilla-test-inference")

# Load train data to recreate vocabularies
train_data = pd.read_csv(
    '/kaggle/input/dakshina/dakshina_dataset_v1.0/hi/lexicons/hi.translit.sampled.train.tsv',
    sep='\t', header=None, dtype=str).dropna()

# Create vocabularies
input_vocab = create_vocab(train_data, 1)  # English
output_vocab = create_vocab(train_data, 0)  # Hindi
inv_input_vocab = {v: k for k, v in input_vocab.items()}
inv_output_vocab = {v: k for k, v in output_vocab.items()}

# Load test data
test_data = pd.read_csv(
    '/kaggle/input/dakshina/dakshina_dataset_v1.0/hi/lexicons/hi.translit.sampled.test.tsv',
    sep='\t', header=None, dtype=str).dropna()
test_dataset = DakshinaDataset(test_data, input_vocab, output_vocab)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, collate_fn=pad_collate)

# Model Config
emb_dim = 64
hidden_dim = 256
num_layers = 3
cell_type = 'LSTM'
dropout = 0.3
beam_size = 3

# Instantiate and Load Model
encoder = Encoder(len(input_vocab), emb_dim, hidden_dim, num_layers, cell_type, dropout)
decoder = Decoder(len(output_vocab), emb_dim, hidden_dim, num_layers, cell_type, dropout)
model = Seq2Seq(encoder, decoder, device).to(device)
model.load_state_dict(torch.load('/kaggle/working/best_model.pt', map_location=device, weights_only=True))
model.eval()

# Inference
correct_words = 0
total_words = 0
total_correct_chars = 0
total_chars = 0
predictions = []

with torch.no_grad():
    for src, tgt in test_loader:
        src, tgt = src.to(device), tgt.to(device)
        batch_size = src.size(0)
        preds = model.predict(src, max_len=30, beam_size=beam_size)

        for i in range(batch_size):
            pred_indices = preds[i].tolist()
            if 2 in pred_indices:
                pred_indices = pred_indices[:pred_indices.index(2)]
            pred_str = ''.join([inv_output_vocab.get(idx, '?') for idx in pred_indices])

            tgt_indices = tgt[i, 1:].tolist()
            if 2 in tgt_indices:
                tgt_indices = tgt_indices[:tgt_indices.index(2)]
            tgt_str = ''.join([inv_output_vocab.get(idx, '?') for idx in tgt_indices])

            input_indices = src[i].tolist()
            input_str = ''.join([inv_input_vocab.get(idx, '?') for idx in input_indices if idx not in [0, input_vocab['<EOS>']]])

            is_correct_word = pred_str == tgt_str
            correct_words += is_correct_word
            total_words += 1

            correct_chars_sample = sum(1 for p, t in zip(pred_str, tgt_str) if p == t)
            total_correct_chars += correct_chars_sample
            total_chars += len(tgt_str)

            predictions.append({
                'input': input_str,
                'target': tgt_str,
                'prediction': pred_str,
                'correct_word': is_correct_word,
                'correct_chars': correct_chars_sample,
                'total_chars': len(tgt_str)
            })

# Accuracies
word_accuracy = (correct_words / total_words) * 100
char_accuracy = (total_correct_chars / total_chars) * 100 if total_chars > 0 else 0

# Log metrics to Wandb
wandb.log({
    "test_word_accuracy": word_accuracy,
    "test_char_accuracy": char_accuracy
})

print(f"\nTest Word Accuracy: {word_accuracy:.2f}%")
print(f"Test Character Accuracy: {char_accuracy:.2f}%")

# Convert predictions to DataFrame
df_predictions = pd.DataFrame(predictions)

# Save All Predictions - CSV (no color)
csv_path = '/kaggle/working/predictions_vanilla.csv'
df_predictions.to_csv(csv_path, index=False)
print(f"CSV saved to {csv_path}")

# Save All Predictions - HTML (no color)
all_html_path = '/kaggle/working/predictions_all.html'
df_predictions.to_html(all_html_path, index=False)
print(f"All HTML (no color) saved to {all_html_path}")

# Color Function for Sample
def highlight_row(row):
    color = 'background-color: #d4edda;' if row['correct_word'] else 'background-color: #f8d7da;'
    return [color] * len(row)

# Save Sample Predictions - HTML (colored)
sample_df = df_predictions.sample(n=min(10, len(df_predictions)), random_state=42)
styled_sample = sample_df.style.apply(highlight_row, axis=1)\
    .set_table_styles([{'selector': 'th', 'props': [('text-align', 'center')]}])\
    .set_properties(**{
        'text-align': 'left',
        'padding': '8px',
        'font-size': '14px',
        'border': '1px solid #ccc'
    }).hide(axis="index")

sample_html_path = '/kaggle/working/predictions_sample_colored.html'
with open(sample_html_path, 'w', encoding='utf-8') as f:
    f.write(f"<h3>Sample Predictions (Color-Coded)</h3>\n{styled_sample.to_html()}")
print(f"Sample colored HTML saved to {sample_html_path}")

# Log to Wandb
artifact = wandb.Artifact('predictions_vanilla', type='predictions')
artifact.add_file(csv_path)
artifact.add_file(all_html_path)
artifact.add_file(sample_html_path)
wandb.log_artifact(artifact)

# Display Sample in Notebook
display(HTML("<h3>Vanilla Sample Predictions (Color-Coded)</h3>"))
display(styled_sample)

# Finish Wandb
wandb.finish()


## Setup For Devanagari Lipi

In [10]:
!fc-list | grep Devanagari

In [11]:
!wget https://noto-website-2.storage.googleapis.com/pkgs/NotoSansDevanagari-hinted.zip
!unzip NotoSansDevanagari-hinted.zip

--2025-05-19 16:03:53--  https://noto-website-2.storage.googleapis.com/pkgs/NotoSansDevanagari-hinted.zip
Resolving noto-website-2.storage.googleapis.com (noto-website-2.storage.googleapis.com)... 142.250.125.207, 173.194.194.207, 142.251.184.207, ...
Connecting to noto-website-2.storage.googleapis.com (noto-website-2.storage.googleapis.com)|142.250.125.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6201435 (5.9M) [application/zip]
Saving to: ‘NotoSansDevanagari-hinted.zip’

NotoSansDevanagari- 100%[===================>]   5.91M  --.-KB/s    in 0.09s   

2025-05-19 16:03:54 (66.5 MB/s) - ‘NotoSansDevanagari-hinted.zip’ saved [6201435/6201435]

Archive:  NotoSansDevanagari-hinted.zip
  inflating: LICENSE_OFL.txt         
  inflating: NotoSansDevanagari-Black.ttf  
  inflating: NotoSansDevanagari-Bold.ttf  
  inflating: NotoSansDevanagari-Condensed.ttf  
  inflating: NotoSansDevanagari-CondensedBlack.ttf  
  inflating: NotoSansDevanagari-CondensedBold.ttf  

In [17]:
import os
import matplotlib.pyplot as plt
from matplotlib import font_manager, rcParams

# Use 'Agg' backend for non-GUI environments (optional but safe)
import matplotlib
matplotlib.use("Agg")

# Path to the Noto Sans Devanagari font (adjust if needed)
font_path = "/kaggle/input/notosans-devanagiri/static/NotoSansDevanagari-Regular.ttf"

if os.path.exists(font_path):
    font_manager.fontManager.addfont(font_path)
    dev_font = font_manager.FontProperties(fname=font_path)
    rcParams['font.family'] = dev_font.get_name()
    print(f"✅ Loaded font: {dev_font.get_name()}")
else:
    print("❌ Devanagari font not found. Falling back to default.")
    dev_font = None

# Test plot to render "भारत"
plt.figure(figsize=(6, 2))
plt.text(0.5, 0.5, "भारत", fontsize=30, ha='center', fontproperties=dev_font)
plt.title("Test: Devanagari Font Rendering", fontproperties=dev_font)
plt.axis('off')
plt.tight_layout()
plt.savefig("/kaggle/working/devanagari_test.png")
plt.show()


✅ Loaded font: Noto Sans Devanagari


In [25]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
import matplotlib.font_manager as fm
import wandb

# Initialize WandB (if not already initialized)
wandb.init(project="DL_ASSIGNMENT_3_RNN", name="char-wise-confusion", reinit=True)

# Font for Hindi (Devanagari)
font_path = "/kaggle/input/notosans-devanagiri/static/NotoSansDevanagari-Regular.ttf"
prop = fm.FontProperties(fname=font_path)
plt.rcParams['font.family'] = prop.get_name()

# Sample 10 rows from the predictions DataFrame
sample_df = df_predictions.sample(n=10, random_state=42)

# Set up a 4x3 grid of plots
fig, axes = plt.subplots(4, 3, figsize=(18, 20))
axes = axes.flatten()

for idx, row in enumerate(sample_df.itertuples()):
    target = list(row.target)
    pred = list(row.prediction)

    # Pad shorter sequence with "_" to make lengths match
    max_len = max(len(target), len(pred))
    target += ['_'] * (max_len - len(target))
    pred += ['_'] * (max_len - len(pred))

    labels = sorted(set(target + pred))
    cm = confusion_matrix(target, pred, labels=labels)

    ax = axes[idx]
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',  # Changed colormap here
                xticklabels=labels, yticklabels=labels, ax=ax, cbar=True,
                annot_kws={"size": 10, "weight": 'bold', "color": "black"})

    # Add titles with corresponding word strings
    ax.set_title(f"Sample {idx + 1}\nTarget: {row.target}\nPred: {row.prediction}",
                 fontproperties=prop, fontsize=10, color='black')
    ax.set_xlabel("Predicted", fontproperties=prop, color='black')
    ax.set_ylabel("Target", fontproperties=prop, color='black')
    ax.tick_params(axis='x', rotation=45, labelsize=9)
    ax.tick_params(axis='y', rotation=0, labelsize=9)

# Hide any unused subplots
for j in range(len(sample_df), len(axes)):
    fig.delaxes(axes[j])

# Save figure
plt.tight_layout()
char_confusion_path = '/kaggle/working/van_confusion.png'
plt.savefig(char_confusion_path, dpi=300)
print(f"Character-wise confusion matrix grid saved to {char_confusion_path}")

# Log to WandB
wandb.log({"char_confusion_matrix_grid": wandb.Image(char_confusion_path)})


Character-wise confusion matrix grid saved to /kaggle/working/van_confusion.png


## Model with attention